In [ ]:
import pennylane as qml
from pennylane import numpy as np
from qiskit.providers.fake_provider import *
from pennylane.transforms import mitigate_with_zne, richardson_extrapolate, fold_global
def configured_backend():
    return FakeManila()

def ansatz(params):
    qml.RY(params[0], wires=0)
def CA():
    qml.CZ(wires=(1, 0))
def U_b():
    qml.Hadamard(wires=0)
dev_noisy = qml.device("qiskit.remote", wires = 2, backend=configured_backend())
dev = qml.device("default.qubit", wires = 2, shots = 1000)

def circuit(params, part=None, j=-1):
    qml.Hadamard(1)
    if part == "Im":
        qml.PhaseShift(-np.pi/2, wires=1)

    ansatz(params)
    CA()
    U_b()

    if j != -1:
        qml.CZ(wires=[1, j])

    U_b()
    CA()

    qml.Hadamard(wires=1)

    return qml.expval(qml.PauliZ(wires=1))

# circuit = qml.QNode(circuit, dev)
circuit = qml.QNode(circuit, dev_noisy)
circuit = mitigate_with_zne(circuit, [1, 2, 3], fold_global, richardson_extrapolate)
def mu(params, j):
    mu_real = circuit(params, part="Re", j=j)
    mu_imag = circuit(params, part="Im", j=j)
    
    return mu_real + 1.0j * mu_imag
def cost_fun(params):
    mu_sum = abs(mu(params, 0))
    norm = abs(mu(params, -1))

    res = 0.5 - 0.5 * mu_sum / (1 * norm)

    return res
lr = 0.01
opt = qml.GradientDescentOptimizer(lr)
from IPython.display import clear_output
w = np.random.randn(1, 
requires_grad=True)
cost_history = []

for it in range(100):
    w, cost = opt.step_and_cost(cost_fun, w)
    clear_output(wait=True)
    print("Step {:3d}       Cost_L = {:9.7f}".format(it, cost), flush=True)
    cost_history.append(cost)

import matplotlib.pyplot as plt
plt.style.use("seaborn")
# plt.plot(np.log(cost_history), "g")
plt.plot(cost_history, "g")
plt.ylabel("Cost function")
plt.xlabel("Optimization steps")
plt.show()